# 📘 Example: Projections in FloodAdapt

In FloodAdapt,  a [Projection](../../api_ref/objects/Projection.qmd) object is used to describe future climate and socio-economic conditions. 

Like all FloodAdapt objects, the Projection has a unique `name` and an optional `description` field. Then main two fields of a Projection are  the:
- `physical_projection`: a [PhysicalProjection](../../api_ref/objects/PhysicalProjection.qmd) object that describes the physical changes in the environment, such as sea level rise and subsidence.
- `socio_economic_change`: a [SocioEconomicChange](../../api_ref/objects/SocioEconomicChange.qmd) object that describes the socio-economic changes, such as population and economic growth.

If you want to learn more about Projections, you can check the [Projections](../../4_user_guide/projections) section of the FloodAdapt GUI documentation.


In this notebook, we will look into all the available FloodAdapt methods to create, save, edit and delete Projections.

## Import libraries

First the required python libraries for this notebook are imported. 

In [ ]:
from flood_adapt import FloodAdapt
from flood_adapt.objects.projections.projections import PhysicalProjection, SocioEconomicChange, Projection
from flood_adapt.objects.forcing.unit_system import UnitTypesLength, UnitfulLength
from pathlib import Path
from flood_adapt.config.config import Settings
import pandas as pd
import geopandas as gpd

## 🚀 **Step 1:** Reading-in the FloodAdapt database

Then, we need to create a `FloodAdapt` object, with the example database of Charleston. This object has all the required methods for adding, copying, editing or deleting projections from the database. 

In [ ]:
# Setup FloodAdapt
STATIC_DATA_DIR = Path("../../_data/examples/static-data").resolve()
settings = Settings(
    DATABASE_ROOT=Path("../../_data/examples").resolve(),
    DATABASE_NAME="charleston_test"
)
fa = FloodAdapt(database_path=settings.database_path)

## 🔎 **Step 2:** Getting available projections from the database

Using the `get_projections()` method of the `FloodAdapt` class, we can get a dictionary of all the projections in the database. The keys of the returned dictionary are the names, descriptions, paths and last modification dates of the projections. We can use this method to check which projections are currently available in the database.

In [ ]:
pd.DataFrame(fa.get_projections()) # here we turn the dictionary to a Pandas DataFrame for better visualization

As can be seen above, right now, there is only one projection available in the database named "**current**". This is a default projection created when the database is created, to describe the current conditions without any future changes.

We can get the [Projection](https://deltares-research.github.io/FloodAdapt/api_ref/objects/Projection.html#flood_adapt.objects.Projection) object of a projection by using the `get_projection()` method of the `FloodAdapt` class. This method takes only the name of the projection as an argument.

In [ ]:
fa.get_projection("current")

As can be seen the "**current**" projection has default values for the attributes of the `physical_projection` and `socio_economic_change`, which essentially means that this projection describes the current conditions without any change.

## 📈 **Step 3:** Creating a new Projection object


A `Projection` object can be created by using the individual FloodAdapt object classes. This can ensure correct type hinting and avoid errors. 

### 🌊🌧️ Physical Projection
First, let's a create a [PhysicalProjection](../api_ref/objects/PhysicalProjection.qmd) that describes future conditions with 0.5 meters of sea level rise. To do this we can use the `sea_level_rise` attribute which has a value with a unit, which in FloodAdapt can be defined using a [UnitfulLength](../api_ref/UnitfulLength.qmd) object with the `value` and `unit` fields. The `value` field is a float and the `unit` field can be one of the [UnitTypesLength](../api_ref/UnitTypesLength.qmd). 

In [ ]:
phys_proj = PhysicalProjection(sea_level_rise=UnitfulLength(value=0.5, units=UnitTypesLength.meters))
phys_proj

### 👥💰 Socio-Economic Change
Then we create a [SocioEconomicChange](../api_ref/objects/SocioEconomicChange.qmd) object that describes future conditions with a population growth of 10% and an economic growth of 5% for the existing areas. To do this we can use the `population_growth` and `economic_growth` attributes which are both floats given in percentages.

In [ ]:
se_change = SocioEconomicChange(population_growth_existing=10,
                          economic_growth=5)
se_change

Now, we can create a `Projection` object, giving it a unique name (which cannot contain any spaces or special characters), and the previously created `PhysicalProjection` and `SocioEconomicChange` objects. The `description` field is optional, and can be used to provide a more extensive description of the projection.

In [ ]:
future_1 = Projection(name="future_1", 
                      description="0.5 m sea level rise, 10% population growth, 5% economic growth", 
                      physical_projection=phys_proj, 
                      socio_economic_change=se_change
                      )
future_1

Alternatively,  a new projection can be created by using the `create_projection()` method of the `FloodAdapt` class. This method takes a single argument which is a dictionary containing the required projection parameters. Let's create the exact same projection as before with this method.

In [ ]:
# Create dictionary
future_1_dict = {
    "name": "slr_50cm",
    "description": "0.5 m sea level rise",
    "physical_projection": {"sea_level_rise": {"value": 0.5, "units": "meters"}},
    "socio_economic_change": {"population_growth_existing": 10, "economic_growth": 5}
}
# Create Projection object from dictionary
future_1_from_dict = fa.create_projection(future_1_dict)

We can now verify that the two projections are identical:

In [ ]:
# Check if the two objects are equal
future_1_from_dict == future_1

## 💾 **Step 4:** Saving a new Projection to the database

In the previous step we have created a `Projection` object, but we have not yet saved it in our database. The `save_projection()` method of the `FloodAdapt` class can be used to achieve that. This method takes a single argument which is a `Projection` object. If a projection with the same name already exists in the database, an error will be raised. Let's save the projection we just created to the database.

In [ ]:
fa.save_projection(future_1)

Using the `get_projections()` method of the `FloodAdapt` class, we can check that the projection has been saved to the database.

In [ ]:
pd.DataFrame(fa.get_projections())

## **Step 5:** Copying and Editing a Projection in the database

If we want to edit small parts of a projection, it is easier to copy an existing projection and edit the copy. This way we do not have to create a new projection from scratch.

A projection can be copied in the database by using the `copy_projection()` method of the `FloodAdapt` class. This method takes three arguments: the name of the projection to be copied and the name and description of the new projection. Let's copy the projection we just created, having in mind that we want to make a projection for 1 meter of sea level rise. 

In [ ]:
# TODO why is it called old_name? we are not replacing the name. This should change to name
fa.copy_projection(old_name="slr_50cm", new_name="slr_1m", new_description="1 m sea level rise")

We can see that now a new projection with name "slr_1m" has been created in the database. However, the actual attributes of the projection are still the same as the original projection. 

In [ ]:
slr_1m = fa.get_projection("slr_1m")
slr_1m.physical_projection.sea_level_rise

We can directly edit the relevant attributes of the projection object. In this case, we want to change the sea level rise to 1 meter.

In [ ]:
slr_1m.physical_projection.sea_level_rise.value = 1.0

Then using the `edit_projection()` method of the `FloodAdapt` class, we can save the changes to the database. This method takes a single argument which is a `Projection` object. The `name` field of the projection object provided will be used to identify which projection is going to be updated in the database, with the given Projection object attributes.

In [ ]:
# TODO do we need an edit?
fa.edit_projection(slr_1m)

Now we can verify that the projection has been updated in the database. The sea level rise is now 1 meter.

In [ ]:
slr_1m = fa.get_projection("slr_1m")
slr_1m.physical_projection.sea_level_rise

## **Step 6:** Deleting a Projection from the database

We now have 3 projections in the database.

In [ ]:
pd.DataFrame(fa.get_projections())


If we want to delete a projection from the database, we can use the `delete_projection()` method of the `FloodAdapt` class. This method takes a single argument which is the name of the projection to be deleted. Let's delete the projection we just created.

In [ ]:
fa.delete_projection("slr_1m")

We can check that the projection has been indeed deleted from the database by using the `get_projections()` method of the `FloodAdapt` class. The projection with name "slr_1m" is no longer in the database.

In [ ]:
pd.DataFrame(fa.get_projections())

## **Step 7:** Use of Sea Level Rise Scenarios

# TODO explain how to add scenarios!
A FloodAdapt database can include sea level rise scenarios, describing a timeline of future sea level rise relative to a reference year. If these scenarios are available in the database the `get_slr_scn_names()` method  will return a list of the available scenarios.

In [ ]:
fa.get_slr_scn_names()

In [ ]:
# TODO the display is not working in the notebook. We need to check why this is the case.
from IPython.display import IFrame, display
from pathlib import Path

# Ensure the path is correct and accessible
html_path = fa.plot_slr_scenarios()

display(IFrame(src=html_path, width='500px', height='200px'))

Then the `interp_slr()` method can be used to interpolate the sea level rise for a given year. This method takes two arguments: the name of the scenario and the year for which we want to interpolate the sea level rise. The method returns a float value representing the interpolated sea level rise in length units of the database.

In [ ]:
# TODO why this is unitless?
fa.interp_slr(slr_scenario="ssp585", year=2050)

# 8. Population Growth in New Developments Areas

If the `population_growth_new` field of a projection is not zero, then the `new_development_elevation` and `new_development_shapefile` should be provided to describe the area that the new development will take place. The `new_development_elevation` field is a [UnitfulLength](https://deltares-research.github.io/FloodAdapt/api_ref/UnitfulLength.html) object which describes the elevation of the new development area. The `new_development_shapefile` field is a [Shapefile](https://deltares-research.github.io/FloodAdapt/api_ref/objects/Shapefile.html#flood_adapt.objects.Shapefile) object which describes the shapefile of the new development area.

In [ ]:
# TODO why call it population_growth_existing? No need to
# TODO add documentation for units
# TODO if population_growth_new is given, shapefile should be mandatory 
proj_2050 = Projection(name="proj_2050",
                    description="2050 projection", 
                    physical_projection= PhysicalProjection(sea_level_rise=UnitfulLength(value=fa.interp_slr(slr_scenario="ssp585", year=2050), units=UnitTypesLength.meters)), 
                    socio_economic_change=SocioEconomicChange(population_growth_existing=10,
                                                              economic_growth=5)
                    )
proj_2050

Using the `get_building_geometries()` method of the `FloodAdapt` class, we can get a geopandas GeoDataFrame of the centroids of the buildings in the database, along with their attributes.

In [ ]:
fa.get_building_geometries().explore() # we use the explore method to make the interactive map

Then we can use a polygon shapefile to define the location of the new development areas. 

In [ ]:

new_dev = gpd.read_file("data/new_dev.geojson")
new_dev.explore()

In [ ]:
proj_2050.socio_economic_change = SocioEconomicChange(population_growth_existing=10,
                                                              economic_growth=5,
                                                              population_growth_new=5,
                                                              new_development_shapefile="data/new_dev.geojson")

In [ ]:
fa.save_projection(proj_2050)

In [ ]:
fa.get_projection("proj_2050")